In [1]:
import numpy as np
import cv2

In [2]:
# resize image using bicubic interpolation
def resize_bicubic_1(image, scale):
    # empty list to store resized rgb channel image
    image_rgb = []

    B = np.array([[-1, 1, -1, 1], [0, 0, 0, 1], [1, 1, 1, 1], [8, 4, 2, 1]])
    B_inv = np.linalg.inv(B)
    B_inv_T = B_inv.T

    counter=0

    # perform bicubic interpolation for color channel (RGB)
    for i in range(3):
        image_channel = image[:, :, i]

        # size of original image
        n_row_in, n_column_in = image_channel.shape

        # size of output image after rescaling
        n_row_out = n_row_in * scale
        n_column_out = n_column_in * scale

        # allocate space for output image
        image_output = np.zeros((n_row_out, n_column_out), dtype=np.uint8)

        # pad original image to make sure every pixel in the original image can find 16 data points for interpolation
        top_row = image_channel[0, :]
        bottom_row = image_channel[-1, :]
        image_row_padding = np.vstack((top_row, image_channel))
        image_row_padding = np.vstack((image_row_padding, bottom_row))
        image_row_padding = np.vstack((image_row_padding, bottom_row))

        leftmost_column = image_row_padding[:, 0]
        rightmost_column = image_row_padding[:, -1]
        image_padding = np.c_[leftmost_column, image_row_padding, rightmost_column, rightmost_column]

        # interpolate image pixel by pixel
        for x in range(n_row_out):
            # corresponding x position in the original image
            x_ori = (x / n_row_out) * n_row_in

            # interpolated value of x (eg: x_ori=1.5, then x_interp=0.5, 0.5 from point (0, 0))
            x_interp = x_ori - np.floor(x_ori)

            # x position in padded image (add one row on top, one column on left, two row on bottom and two column on right)
            # we need to add 1 to recover the original x position in the original image without padding
            #x_padding = int(x_ori + 1)
            x_padding = int(np.floor(x_ori) + 1)

            for y in range(n_column_out):
                # correspond y postition in the original image
                y_ori = (y / n_column_out) * n_column_in

                # interppolated value of y
                y_interp = y_ori - np.floor(y_ori)

                # (1, 1) position of the image matrix after padding correspond to (0, 0) position of the original image matrix
                #y_padding = int(y_ori + 1)
                y_padding = int(np.floor(y_ori) + 1)

                # when interpolate point can be found in the original image matrix
                # then there is no need to perform bicubic interpolation for that point
                # we can simply copy the pixel value of that poibt from orginal image to scaled image
                if x_interp==0.0 and y_interp==0.0:
                    # there should always be (original image width * origianl image height) times we can copy the pixel value like this
                    # for three rgb channel we can enter into this if statement for 3 * (original image width * origianl image height) times
                    #counter += 1
                    image_output[x][y] = image_channel[int(x_ori)][int(y_ori)]
                else:
                    #interpolate value in x direction (row vector)
                    X = np.expand_dims(np.array([x_interp**3, x_interp**2, x_interp**1, x_interp**0]), axis=0)
                    # interpolate value in y direction (column vector)
                    Y = np.expand_dims(np.array([y_interp**3, y_interp**2, y_interp**1, y_interp**0]), axis=1)

                    F = np.array([[image_padding[x_padding-1][y_padding-1], image_padding[x_padding-1][y_padding], image_padding[x_padding-1][y_padding+1], image_padding[x_padding-1][y_padding+2]],
                                    [image_padding[x_padding][y_padding-1], image_padding[x_padding][y_padding], image_padding[x_padding][y_padding+1], image_padding[x_padding][y_padding+2]],
                                    [image_padding[x_padding+1][y_padding-1], image_padding[x_padding+1][y_padding], image_padding[x_padding+1][y_padding+1], image_padding[x_padding+1][y_padding+2]],
                                    [image_padding[x_padding+2][y_padding-1], image_padding[x_padding+2][y_padding], image_padding[x_padding+2][y_padding+1], image_padding[x_padding+2][y_padding+2]]])

                    # bicubic interpolation
                    value = X.dot(B_inv).dot(F).dot(B_inv_T).dot(Y)

                    # if value < 0 or value > 255:
                    #     print(value)

                    # after bicubic interpolation between adjacent pixels the floats are returned instead of integers between 0 and 255
                    # that's why we need to clamp the value between 0 and 255 in order to show image matrix as a image
                    if value < 0:
                        value = 0
                    elif value > 255:
                        value = 255

                    image_output[x][y] = value

        # insert resized rgb channel in to a list
        image_rgb.append(image_output)

    image_rgb_output = cv2.merge((image_rgb[0], image_rgb[1], image_rgb[2]))
    #print(counter)
    
    return image_rgb_output

            

In [3]:
def cubic(p0, p1, p2, p3, x):
    a = -1/2 * p0 + 3/2 * p1 - 3/2 * p2 + 1/2 * p3
    b = p0 - 5/2 * p1 + 2 * p2 - 1/2 * p3
    c = -1/2 * p0 + 1/2 * p2
    d = p1

    return a * x**3 + b * x**2 + c * x**1 + d

def resize_bicubic_2(image, scale):
    # empty list to store resized rgb channel image
    image_rgb = []
    
    # perform bicubic interpolation on each color channel (RGB)
    for i in range(3):

        image_channel = image[:, :, i]
       
        # size of original image channel
        n_row_in, n_column_in = image_channel.shape

        # size of output image after rescaling
        n_row_out = n_row_in * scale
        n_column_out = n_column_in * scale

        # allocate space for output image
        image_output = np.zeros((n_row_out, n_column_out), dtype=np.uint8)

        # pad original image to make sure every pixel in the original image can find 16 data points for interpolation
        top_row = image_channel[0, :]
        bottom_row = image_channel[-1, :]
        image_row_padding = np.vstack((top_row, image_channel))
        image_row_padding = np.vstack((image_row_padding, bottom_row))
        image_row_padding = np.vstack((image_row_padding, bottom_row))

        leftmost_column = image_row_padding[:, 0]
        rightmost_column = image_row_padding[:, -1]
        image_padding = np.c_[leftmost_column, image_row_padding, rightmost_column, rightmost_column]

        for x in range(n_row_out):
            x_ori = (x / n_row_out) * n_row_in
            x_int = int(x_ori+1)
            x_interp = x_ori - np.floor(x_ori)
            
            for y in range(n_column_out):
                y_ori = (y / n_column_out) * n_column_in
                y_int = int(y_ori+1)
                y_interp = y_ori - np.floor(y_ori)

                # bicubic interpolation
                # fix y vary x
                a = cubic(image_padding[x_int-1][y_int-1], image_padding[x_int][y_int-1], image_padding[x_int+1][y_int-1], image_padding[x_int+2][y_int-1], x_interp)
                b = cubic(image_padding[x_int-1][y_int], image_padding[x_int][y_int], image_padding[x_int+1][y_int], image_padding[x_int+2][y_int], x_interp)
                c = cubic(image_padding[x_int-1][y_int+1], image_padding[x_int][y_int+1], image_padding[x_int+1][y_int+1], image_padding[x_int+2][y_int+1], x_interp)
                d = cubic(image_padding[x_int-1][y_int+2], image_padding[x_int][y_int+2], image_padding[x_int+1][y_int+2], image_padding[x_int+2][y_int+2], x_interp)

                # fix x vary y
                value = cubic(a, b, c, d, y_interp)
                if value < 0:
                    value = 0
                elif value > 255:
                    value = 255
                image_output[x][y] = value

        image_rgb.append(image_output)

    image_rgb_output = cv2.merge((image_rgb[0], image_rgb[1], image_rgb[2]))
    
    return image_rgb_output

In [4]:
# resize image using bicubic interpolation
def resize_bicubic_3(image, scale):
    # empty list to store resized rgb channel image
    image_rgb = []

    B = np.array([[-1, 1, -1, 1], [0, 0, 0, 1], [1, 1, 1, 1], [8, 4, 2, 1]])
    B_inv = np.linalg.inv(B)
    B_inv_T = B_inv.T

    counter=0

    # perform bicubic interpolation for color channel (RGB)
    for i in range(3):
        image_channel = image[:, :, i]

        # size of original image
        n_row_in, n_column_in = image_channel.shape

        # size of output image after rescaling
        n_row_out = n_row_in * scale
        n_column_out = n_column_in * scale

        # allocate space for output image
        image_output = np.zeros((n_row_out, n_column_out), dtype=np.uint8)

        # pad original image to make sure every pixel in the original image can find 16 data points for interpolation
        top_row = image_channel[0, :]
        bottom_row = image_channel[-1, :]
        image_row_padding = np.vstack((top_row, image_channel))
        image_row_padding = np.vstack((image_row_padding, bottom_row))
        image_row_padding = np.vstack((image_row_padding, bottom_row))

        leftmost_column = image_row_padding[:, 0]
        rightmost_column = image_row_padding[:, -1]
        image_padding = np.c_[leftmost_column, image_row_padding, rightmost_column, rightmost_column]

        # preallocate space for F matrix
        # F stores f matrix for each pixel in the original image matrix (after padding)
        # each f is a 4 by 4 matrix that stores 16 pixels value for bicubic interpolation
        F = np.zeros((n_row_in, n_column_in, 4, 4))

        for x in range(n_row_in):
            x_padding = x + 1 # reposition x since we are using image matrix after padding
            for y in range(n_column_in):
                y_padding = y + 1 # reposition y since we are using image matrix after padding
                f = np.array([[image_padding[x_padding-1][y_padding-1], image_padding[x_padding-1][y_padding], image_padding[x_padding-1][y_padding+1], image_padding[x_padding-1][y_padding+2]],
                              [image_padding[x_padding][y_padding-1],   image_padding[x_padding][y_padding],   image_padding[x_padding][y_padding+1],   image_padding[x_padding][y_padding+2]],
                              [image_padding[x_padding+1][y_padding-1], image_padding[x_padding+1][y_padding], image_padding[x_padding+1][y_padding+1], image_padding[x_padding+1][y_padding+2]],
                              [image_padding[x_padding+2][y_padding-1], image_padding[x_padding+2][y_padding], image_padding[x_padding+2][y_padding+1], image_padding[x_padding+2][y_padding+2]]])
                F[x][y] = f

        # interpolate image pixel by pixel
        for x in range(n_row_out):
            # corresponding x position in the original image
            x_ori = (x / n_row_out) * n_row_in

            # interpolated value of x (eg: x_ori=1.5, then x_interp=0.5, 0.5 from point (0, 0))
            x_interp = x_ori - np.floor(x_ori)

            # x position in padded image (add one row on top, one column on left, two row on bottom and two column on right)
            # we need to add 1 to recover the original x position in the original image without padding
            #x_padding = int(x_ori + 1)
            #x_padding = int(np.floor(x_ori) + 1)
            x_int = int(np.floor(x_ori))

            for y in range(n_column_out):
                # correspond y postition in the original image
                y_ori = (y / n_column_out) * n_column_in

                # interppolated value of y
                y_interp = y_ori - np.floor(y_ori)

                # (1, 1) position of the image matrix after padding correspond to (0, 0) position of the original image matrix
                #y_padding = int(y_ori + 1)
                #y_padding = int(np.floor(y_ori) + 1)
                y_int = int(np.floor(y_ori))

                # when interpolate point can be found in the original image matrix
                # then there is no need to perform bicubic interpolation for that point
                # we can simply copy the pixel value of that poibt from orginal image to scaled image
                if x_interp==0.0 and y_interp==0.0:
                    # there should always be (original image width * origianl image height) times we can copy the pixel value like this
                    # for three rgb channel we can enter into this if statement for 3 * (original image width * origianl image height) times
                    #counter += 1
                    image_output[x][y] = image_channel[int(x_ori)][int(y_ori)]
                else:
                    #interpolate value in x direction (row vector)
                    X = np.expand_dims(np.array([x_interp**3, x_interp**2, x_interp**1, x_interp**0]), axis=0)
                    # interpolate value in y direction (column vector)
                    Y = np.expand_dims(np.array([y_interp**3, y_interp**2, y_interp**1, y_interp**0]), axis=1)

                    F_interp = F[x_int][y_int]

                    # bicubic interpolation
                    value = X.dot(B_inv).dot(F_interp).dot(B_inv_T).dot(Y)

                    # if value < 0 or value > 255:
                    #     print(value)

                    # after bicubic interpolation between adjacent pixels the floats are returned instead of integers between 0 and 255
                    # that's why we need to clamp the value between 0 and 255 in order to show image matrix as a image
                    if value < 0:
                        value = 0
                    elif value > 255:
                        value = 255

                    image_output[x][y] = value

        # insert resized rgb channel in to a list
        image_rgb.append(image_output)

    image_rgb_output = cv2.merge((image_rgb[0], image_rgb[1], image_rgb[2]))
    
    return image_rgb_output

In [5]:
a = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(a)

top_row = a[0, :]
bottom_row = a[-1, :]
b = np.vstack((top_row, a))
b = np.vstack((b, bottom_row))
b = np.vstack((b, bottom_row))
print(b)
leftmost_column = b[:, 0]
rightmost_column = b[:, -1]
c = np.c_[leftmost_column, b, rightmost_column, rightmost_column]
print(c)

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[1 2 3]
 [1 2 3]
 [4 5 6]
 [7 8 9]
 [7 8 9]
 [7 8 9]]
[[1 1 2 3 3 3]
 [1 1 2 3 3 3]
 [4 4 5 6 6 6]
 [7 7 8 9 9 9]
 [7 7 8 9 9 9]
 [7 7 8 9 9 9]]


In [6]:
# create a 3 by 2 matrix(3 rows 2 columns) and each element within is a 2 by 2 matrix
a = np.zeros((3, 2, 2, 2))
print(a)

[[[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]]


In [13]:
image = cv2.imread('./Image/man.png')

#image_resized = resize_bicubic_1(image, 2)

In [8]:
#image_resized_2 = resize_bicubic_2(image, 2)

In [14]:
image_resized_3 = resize_bicubic_3(image, 16)

In [ ]:
#image_resized_4 = cv2.resize(image, (512, 384), interpolation=cv2.INTER_CUBIC)

In [15]:
#.imshow("Resized", image_resized)
#cv2.imshow("Resized 2", image_resized_2)
cv2.imshow("Resized 3 (Bicubic)", image_resized_3)
#cv2.imshow("Reszied 4", image_resized_4)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1